# Experiments: Fooling Lime


## Imports

In [4]:
from __future__ import print_function

from data_utils import read_data
from log_utils import setup_logger
from train_utils import *
from train_utils import LossConfig

## Loading Data and Classifier

In [2]:
# model_type = "emb-att"
# emb_size = 123
# hid_size = 123
# task_name = 'occupation-classification'
# best_epoch = 1
# seed = 1
# # c_hammer = 0.0
# c_entropy = 0.0
# clip_vocab = True
# block_words = ['he', 'she', 'her', 'his', 'him', 'himself', 'herself', 'hers', 'mr', 'mrs', 'ms', 'mr.', 'mrs.', 'ms.']
#
# train, dev, test, n_words, i2w, i2t, n_tags = read_data(task_name,model_type,clip_vocab, block_words)
#
# model = get_trained_model(model_type, task_name, best_epoch, seed, c_hammer, c_entropy, n_words, emb_size, hid_size, n_tags)

### Test
For three different manipulations: 0.0 (no manipulation), 0.1, 1.0


In [5]:
model_type = "emb-att"
c_entropy = 0.0
clip_vocab = True
block_words = ['he', 'she', 'her', 'his', 'him', 'himself', 'herself', 'hers', 'mr', 'mrs', 'ms', 'mr.', 'mrs.', 'ms.']
seed = 1

c_hammers = [0.0, 0.1, 1.0]
tasks = ['sst-wiki', 'pronoun', 'occupation-classification']
best_epochs = [1, 2, 1]

# for task in tasks:
# for now only for single task
task = tasks[2]

# for c_hammer in c_hammers:
# for now only for single hammer
loss_config = LossConfig(c_hammers[1], 0.0, 0.0)

set_seed(seed)

logger = setup_logger()

_, _, test, vocabulary = read_data(task, model_type, logger, clip_vocab, block_words)

model = get_trained_model(model_type, task, best_epochs[tasks.index(task)], seed, loss_config, vocabulary)

# accuracy, loss = evaluate(model, test, vocabulary, loss_config, logger=logger, stage='test', attn_stats=True, num_vis=100)

# print('Accuracy: ', accuracy)
# print('Loss: ', loss)

Reading data:
	occupation-classification
Reading data:
	occupation-classification
Using block words:
	['he', 'she', 'her', 'his', 'him', 'himself', 'herself', 'hers', 'mr', 'mrs', 'ms', 'mr.', 'mrs.', 'ms.']
Using block words:
	['he', 'she', 'her', 'his', 'him', 'himself', 'herself', 'hers', 'mr', 'mrs', 'ms', 'mr.', 'mrs.', 'ms.']


### Run Lime Explainer on Model

In [9]:
from lime.lime_text import LimeTextExplainer

class_names = ['surgeon', 'non-surgeon']
# class_names = [0, 1]

explainer = LimeTextExplainer(class_names=class_names)

idx = 83
test_instance = test[idx]
target = test_instance[4]

# prediction = model.predict_probabilities(test_instance)
# print('prediction: ', prediction)
# print('target: ', target)

word_indices = test_instance[1]
words = [vocabulary.i2w[idx] for idx in word_indices]
sentence = ' '.join(words)
print(sentence)

model.data_instance_for_prediction(test_instance)

exp = explainer.explain_instance(sentence, model.predict_probabilities, num_features=6)
print('Document ID: %d' % idx)

prediction = model.predict_probabilities([sentence])[0]
print(f'Probability ({class_names[1]}) =', prediction[1])
print(f'Probability ({class_names[0]}) =', prediction[0])

print('True class: %s' % class_names[target])

ms. romeo practices medicine in new hyde park, ny and specializes in pediatric gastroenterology. ms. romeo is affiliated with cohen childrens medical center and north shore university hospital. ms. romeo speaks english, chinese and spanish.
Original sentence:  ms. romeo practices medicine in new hyde park, ny and specializes in pediatric gastroenterology. ms. romeo is affiliated with cohen childrens medical center and north shore university hospital. ms. romeo speaks english, chinese and spanish.
words from lime are equals indices stored in model
softmax dim 0  tensor([4.0808e-07, 1.0000e+00], grad_fn=<SoftmaxBackward>)
Document ID: 83
Original sentence:  ms. romeo practices medicine in new hyde park, ny and specializes in pediatric gastroenterology. ms. romeo is affiliated with cohen childrens medical center and north shore university hospital. ms. romeo speaks english, chinese and spanish.
words from lime are equals indices stored in model
softmax dim 0  tensor([4.0808e-07, 1.0000e+0

In [ ]:
exp.as_list()
